In [342]:
from data_utils import load_data, rolling_window
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import scipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%time X, y, files = load_data('train_1', sample=True)

loading 20 with 7 processes.
Wall time: 8.05 s


In [102]:
X[0].shape

(240000L, 16L)

In [104]:
def slidingWindow(sequence, window , step=1):
    for i in range(0,len(sequence)-window+1,step):
        yield sequence[i:i+window]

In [105]:
chunks = slidingWindow(X[0],400*60,400*60)

In [106]:
test = np.array(list(chunks))

In [108]:
test.shape # 10, 1 minute long chunks, for each chunk lets convert it to the frequency domain

(10L, 24000L, 16L)

In [301]:
from scipy.signal import butter, lfilter


def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut=0.1, highcut=180.0, fs=400.0, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [322]:
filtered = butter_bandpass_filter(X[0])

In [323]:
second_clips = np.array(list(slidingWindow(filtered,400,400)))
# second_clips = second_clips.reshape((600, 16, 400))

In [324]:
# test = np.array([np.log10(np.abs(np.fft.rfft(minute, axis=1))) for minute in  test])

In [325]:
second_clips[0].shape

(400L, 16L)

In [326]:
def fft(time_data):
    return np.log10(np.absolute(np.fft.rfft(time_data, axis=0)[1:48, :]))

In [327]:
test = np.array(map(fft, second_clips))

In [328]:
test.shape

(600L, 47L, 16L)

In [339]:
def upper_right_triangle(matrix):
    accum = []
    for i in range(matrix.shape[0]):
        for j in range(i+1, matrix.shape[1]):
            accum.append(matrix[i, j])

    return np.array(accum)

In [347]:
import sklearn.preprocessing

In [348]:
def freq_corr(fft_data):
    scaled = sklearn.preprocessing.scale(fft_data, axis=1)
    corr_matrix = np.corrcoef(scaled)
    eigenvalues = np.absolute(np.linalg.eig(corr_matrix)[0])
    eigenvalues.sort()
    corr_coefficients = upper_right_triangle(corr_matrix)
    return np.concatenate((corr_coefficients, eigenvalues))

In [356]:
def time_corr(time_data):
    resampled = scipy.signal.resample(time_data, 400, axis=1) if time_data.shape[-1] > 400 else time_data
    scaled = sklearn.preprocessing.scale(resampled, axis=0)
    corr_matrix = np.corrcoef(scaled)
    eigenvalues = np.absolute(np.linalg.eig(corr_matrix)[0])
    corr_coefficients = upper_right_triangle(corr_matrix) # custom func
    return np.concatenate((corr_coefficients, eigenvalues))


In [357]:
def transform(data):
    fft_out = fft(data)
    freq_corr_out = freq_corr(fft_out)
    time_corr_out = time_corr(data)
    return np.concatenate((fft_out.ravel(), freq_corr_out, time_corr_out))


In [360]:
transform(second_clips[0].T).shape

(3372L,)